# Notebook para analisar os dados da base Bolsa Família

## Importando as bibliotecas necessárias

In [64]:
# Importando a sessão do Spark que se comunica com o cluster
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, StringType, DoubleType
from pyspark.sql.functions import regexp_replace, col, to_timestamp

## Iniciando Sessão Spark e importando o arquivo CSV

In [17]:
#Instanciando a sessão spark
spark = SparkSession.builder.appName("Bolsa Familia").getOrCreate()

In [55]:
caminho = "./data/202401_NovoBolsaFamilia.csv"

df = spark.read.csv(path=caminho, 
                    header=True,
                    sep=';',
                    encoding='ISO-8859-1')

df.show()

+---------------+--------------+---+----------------------+--------------------+--------------+--------------+--------------------+-------------+
|MÊS COMPETÊNCIA|MÊS REFERÊNCIA| UF|CÓDIGO MUNICÍPIO SIAFI|      NOME MUNICÍPIO|CPF FAVORECIDO|NIS FAVORECIDO|     NOME FAVORECIDO|VALOR PARCELA|
+---------------+--------------+---+----------------------+--------------------+--------------+--------------+--------------------+-------------+
|         202401|        202303| AC|                  0139|          RIO BRANCO|***.511.972-**|   16291115205|JANAINA FERREIRA ...|       800,00|
|         202401|        202303| BA|                  3329|             ANGUERA|***.909.745-**|   20474429886|TACIANE MARQUES D...|       750,00|
|         202401|        202303| BA|                  3731|         MONTE SANTO|***.872.338-**|   20195376298|MAILSON DOS SANTO...|       650,00|
|         202401|        202303| BA|                  3761|         NOVA VICOSA|***.665.905-**|   12723589082|IRENI RICARDO 

## Ajustando os tipos dos dados

In [56]:
df.printSchema()

root
 |-- MÊS COMPETÊNCIA: string (nullable = true)
 |-- MÊS REFERÊNCIA: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- CÓDIGO MUNICÍPIO SIAFI: string (nullable = true)
 |-- NOME MUNICÍPIO: string (nullable = true)
 |-- CPF FAVORECIDO: string (nullable = true)
 |-- NIS FAVORECIDO: string (nullable = true)
 |-- NOME FAVORECIDO: string (nullable = true)
 |-- VALOR PARCELA: string (nullable = true)



In [77]:
df_cast = df.withColumn(
    'MÊS COMPETÊNCIA', to_timestamp('MÊS COMPETÊNCIA', 'yyyyMM')
).withColumn(
    'MÊS REFERÊNCIA', to_timestamp('MÊS REFERÊNCIA', 'yyyyMM')
).withColumn(
    'NIS FAVORECIDO', col('NIS FAVORECIDO').cast('long')
).withColumn(
    'VALOR PARCELA', regexp_replace('VALOR PARCELA', ',', '.')
).withColumn(
    'VALOR PARCELA', col('VALOR PARCELA').cast('double')
)

In [78]:
df_cast.printSchema()

root
 |-- MÊS COMPETÊNCIA: timestamp (nullable = true)
 |-- MÊS REFERÊNCIA: timestamp (nullable = true)
 |-- UF: string (nullable = true)
 |-- CÓDIGO MUNICÍPIO SIAFI: string (nullable = true)
 |-- NOME MUNICÍPIO: string (nullable = true)
 |-- CPF FAVORECIDO: string (nullable = true)
 |-- NIS FAVORECIDO: long (nullable = true)
 |-- NOME FAVORECIDO: string (nullable = true)
 |-- VALOR PARCELA: double (nullable = true)



In [79]:
df_cast.show()

+-------------------+-------------------+---+----------------------+--------------------+--------------+--------------+--------------------+-------------+
|    MÊS COMPETÊNCIA|     MÊS REFERÊNCIA| UF|CÓDIGO MUNICÍPIO SIAFI|      NOME MUNICÍPIO|CPF FAVORECIDO|NIS FAVORECIDO|     NOME FAVORECIDO|VALOR PARCELA|
+-------------------+-------------------+---+----------------------+--------------------+--------------+--------------+--------------------+-------------+
|2024-01-01 00:00:00|2023-03-01 00:00:00| AC|                  0139|          RIO BRANCO|***.511.972-**|   16291115205|JANAINA FERREIRA ...|        800.0|
|2024-01-01 00:00:00|2023-03-01 00:00:00| BA|                  3329|             ANGUERA|***.909.745-**|   20474429886|TACIANE MARQUES D...|        750.0|
|2024-01-01 00:00:00|2023-03-01 00:00:00| BA|                  3731|         MONTE SANTO|***.872.338-**|   20195376298|MAILSON DOS SANTO...|        650.0|
|2024-01-01 00:00:00|2023-03-01 00:00:00| BA|                  3761|  